## Notebook 9.2 (KNN)

There are many types of classifiers in machine learning. These are algorithms that aim to assign samples to a labeled category based on features of the sample. It is a type of clustering algorithm. One of the simplest forms is called k-Nearest Neighbors. This is a supervised learning algorithm that assigns samples to a category based on the label of the point in the trained data set that is closest to it (or the k-nearest points that are closest to it). 

In [1]:
import numpy as np
import pandas as pd
import toyplot
toyplot.config.autoformat = "html" # plot as html figure or pdf - ghost script needed for pdf

### Simulated data
Although we could use an existing data set, such as the Iris data set for testing out this method, I again prefer to simulate our own data using numpy random. This exercise is good practice for using numpy, and it ensures that you know what the data look like and how they were generated. Let's try to generate data in two clouds of points drawn from different means but with some overlapping points. Points drawn from different means will be labeled by the distribution they were drawn from, and our goal is to label the points as to which distribution they belong to. 

In [2]:
# how many points in each cloud
nsize = 1000

# 2-d cloud of points with mean at [1, 1] and std [1, 1]
np.random.seed(123)
cloud1 = np.random.normal([1, 1], [0.5, 1], (nsize, 2)) # bivariate - mean at (1,1) and SD of 0.5, 1
cloud2 = np.random.normal([2, 2], [1, 0.5], (nsize, 2))

In [3]:
# concatenate all of the points
data = pd.DataFrame({
    'x': np.concatenate([cloud1[:, 0], cloud2[:, 0]]),
    'y': np.concatenate([cloud1[:, 1], cloud2[:, 1]]),
    'label': np.concatenate([np.repeat(0, nsize), np.repeat(1, nsize)]),
})

# shuffle order of rows
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,label,x,y
0,0,0.560232,1.283627
1,0,1.179844,0.635643
2,1,2.181113,2.433217
3,0,1.266755,0.584381
4,1,2.144146,1.953467


In [4]:
toyplot.scatterplot(
    data.x, 
    data.y, 
    opacity=0.5,
    color=[toyplot.color.Palette()[i] for i in data.label], 
    width=300, 
    height=250);

<g class="toyplot-Datum" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:0.5;stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0" transform="translate(159.3593585

### Fit the data using a KNN

In [7]:
# separate data into training and test and format for sklearn
X = data[['x', 'y']].values
y = data.label.values

# test vs train
ntest = 200
Xtest = X[:ntest]
Xtrain = X[ntest:]
ytest = y[:ntest]
ytrain = y[ntest:]

In [9]:
# 1. choose a model
from sklearn.neighbors import KNeighborsClassifier


In [10]:
# 2. initialize and choose hyperparameters
knn = KNeighborsClassifier(n_neighbors=1)


In [11]:
# 3. fit the model
knn.fit(Xtrain, ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [12]:
# 4. predict 
yfit = knn.predict(Xtest)

### Validation

How well did the yfit values match ytest? Does it fit better if we use a different hyperparameter?

In [13]:
# 78% of points were assigned correctly... not too bad
np.mean(yfit == ytest)

0.78000000000000003

In [15]:
# draw uniform points from min to max in 2 dimensions
newdata = np.random.uniform(-2, 6, (nsize, 2))

In [16]:
# plot predictions as background color
c, a, m = toyplot.scatterplot(
    newdata[:, 0],
    newdata[:, 1],
    color=[toyplot.color.Palette()[i] for i in knn.predict(newdata)],
    size=8, 
    opacity=0.5,
    width=350, 
    height=350,
);

# plot test data (not used in training) and show prediction
a.scatterplot(
    Xtest[:, 0], Xtest[:, 1], size=8, 
    mstyle={"stroke": 'black'},
    color=[toyplot.color.Palette()[i] for i in ytest],
);

<g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.5;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" transform="translate(141.68000319999649,

In [18]:
# Cross validation (to try to improve validation)
cvs = np.zeros(200)
for k in range(1, 200):
    knn = KNeighborsClassifier(n_neighbors=k) # craete a new model 
    knn.fit(Xtrain, ytrain) # train it
    yfit = knn.predict(Xtest) # make a prediction
    cvs[k] = np.mean(yfit == ytest)

In [19]:
toyplot.plot(cvs, height=300, width=350, ymin=0.75);

<path d="M 50.0 2125.0000000000009 L 51.25 174.99999999999991 L 52.5 224.99999999999994 L 53.75 149.99999999999986 L 55.0 174.99999999999991 L 56.25 212.49999999999994 L 57.5 174.99999999999991 L 58.75 137.49999999999983 L 60.0 112.4999999999998 L 61.25 137.49999999999983 L 62.5 137.49999999999983 L 63.75 124.99999999999982 L 65.0 99.999999999999801 L 66.25 62.500000000000021 L 67.5 50.0 L 68.75 99.999999999999801 L 70.0 87.500000000000043 L 71.25 87.500000000000043 L 72.5 99.999999999999801 L 73.75 75.000000000000043 L 75.0 87.500000000000043 L 76.25 87.500000000000043 L 77.5 62.500000000000021 L 78.75 75.000000000000043 L 80.0 50.0 L 81.25 75.000000000000043 L 82.5 87.500000000000043 L 83.75 112.4999999999998 L 85.0 62.500000000000021 L 86.25 75.000000000000043 L 87.5 75.000000000000043 L 88.75 112.4999999999998 L 90.0 75.000000000000043 L 91.25 75.000000000000043 L 92.5 50.0 L 93.75 75.000000000000043 L 95.0 62.500000000000021 L 96.25 75.000000000000043 L 97.5 62.500000000000021 L 98.75 75.000000000000043 L 100.0 75.000000000000043 L 101.25 62.500000000000021 L 102.5 50.0 L 103.75 75.000000000000043 L 105.0 75.000000000000043 L 106.25 62.500000000000021 L 107.5 50.0 L 108.75 75.000000000000043 L 110.0 62.500000000000021 L 111.25 75.000000000000043 L 112.5 75.000000000000043 L 113.75 87.500000000000043 L 115.0 75.000000000000043 L 116.25 87.500000000000043 L 117.5 112.4999999999998 L 118.75 112.4999999999998 L 120.00000000000001 112.4999999999998 L 121.25 99.999999999999801 L 122.5 99.999999999999801 L 123.75 99.999999999999801 L 125.0 112.4999999999998 L 126.25 99.999999999999801 L 127.5 87.500000000000043 L 128.75 112.4999999999998 L 130.0 99.999999999999801 L 131.25 87.500000000000043 L 132.5 99.999999999999801 L 133.75 112.4999999999998 L 135.0 124.99999999999982 L 136.25 124.99999999999982 L 137.5 99.999999999999801 L 138.75 99.999999999999801 L 140.0 124.99999999999982 L 141.25 99.999999999999801 L 142.5 99.999999999999801 L 143.75 99.999999999999801 L 145.0 124.99999999999982 L 146.25 99.999999999999801 L 147.5 87.500000000000043 L 148.75 87.500000000000043 L 150.0 99.999999999999801 L 151.25 87.500000000000043 L 152.5 87.500000000000043 L 153.75 87.500000000000043 L 155.0 87.500000000000043 L 156.25 99.999999999999801 L 157.5 87.500000000000043 L 158.75 87.500000000000043 L 160.0 75.000000000000043 L 161.25 75.000000000000043 L 162.5 87.500000000000043 L 163.75 62.500000000000021 L 165.0 75.000000000000043 L 166.25 87.500000000000043 L 167.5 99.999999999999801 L 168.75 99.999999999999801 L 170.0 87.500000000000043 L 171.25 99.999999999999801 L 172.5 99.999999999999801 L 173.75 87.500000000000043 L 175.0 99.999999999999801 L 176.25 87.500000000000043 L 177.5 87.500000000000043 L 178.75 87.500000000000043 L 180.0 99.999999999999801 L 181.25 99.999999999999801 L 182.5 99.999999999999801 L 183.75 99.999999999999801 L 185.0 99.999999999999801 L 186.25 112.4999999999998 L 187.5 99.999999999999801 L 188.75000000000003 99.999999999999801 L 190.00000000000003 112.4999999999998 L 191.24999999999997 87.500000000000043 L 192.49999999999997 87.500000000000043 L 193.75 87.500000000000043 L 195.0 87.500000000000043 L 196.25 87.500000000000043 L 197.5 112.4999999999998 L 198.75 124.99999999999982 L 200.0 112.4999999999998 L 201.25 99.999999999999801 L 202.5 112.4999999999998 L 203.75 75.000000000000043 L 205.0 62.500000000000021 L 206.25 75.000000000000043 L 207.5 99.999999999999801 L 208.75 75.000000000000043 L 210.0 87.500000000000043 L 211.25 87.500000000000043 L 212.5 87.500000000000043 L 213.75 87.500000000000043 L 215.0 99.999999999999801 L 216.25 112.4999999999998 L 217.5 99.999999999999801 L 218.75 87.500000000000043 L 220.00000000000003 99.999999999999801 L 221.25000000000003 99.999999999999801 L 222.49999999999997 99.999999999999801 L 223.74999999999997 87.500000000000043 L 225.0 99.999999999999801 L 226.25 99.999999999999801 L 227.5 124.99999999999982 L 228.75 112.4999999999998 L 230.0 137.49999999999983 L 231.25 99.99999

In [40]:
# the best hyperparameter - 14 best neighbors makes the best prediction 
cvs.argmax()

14

### Using best fit model
This looks a lot better than the one above, and scores higher too. 

In [20]:
# train the new model on the best cross-val hyperparameter
knn = KNeighborsClassifier(n_neighbors=cvs.argmax())
knn.fit(Xtrain, ytrain)
yfit = knn.predict(Xtest)
print("score:", np.mean(yfit == ytest))

# plot predictions as background color
c, a, m = toyplot.scatterplot(
    newdata[:, 0],
    newdata[:, 1],
    color=[toyplot.color.Palette()[i] for i in knn.predict(newdata)],
    size=8, 
    opacity=0.5,
    width=350, 
    height=350,
);

# plot test data (not used in training) and show prediction
a.scatterplot(
    Xtest[:, 0], Xtest[:, 1], size=8, 
    mstyle={"stroke": 'black'},
    color=[toyplot.color.Palette()[i] for i in ytest],
);

score: 0.83


<g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);f

In [ ]:
# apply machine learning technique to GBIF dataframe 
# restrcutring data and validating it 
# ie. which samples are usually found in which state?